## Start Spark session

In [153]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.0",
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1531460095132_0010,spark,idle,Link,Link,✔


SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1531460095132_0010,spark,idle,Link,Link,✔


In [154]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.streaming.Trigger

import org.apache.spark.sql.streaming.Trigger

## Drop topics

In [155]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --delete --topic rawevents --zookeeper "zk0-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk1-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk3-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181"
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --delete --topic streamPerHourPerDay --zookeeper "zk0-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk1-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk3-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181"
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --delete --topic streamPerHour --zookeeper "zk0-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk1-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk3-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181"
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --delete --topic streamPerHourAlert2 --zookeeper "zk0-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk1-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk3-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181"

Topic rawevents is marked for deletion.
Note: This will have no impact if delete.topic.enable is not set to true.
Topic streamPerHourPerDay is marked for deletion.
Note: This will have no impact if delete.topic.enable is not set to true.
Topic streamPerHour is marked for deletion.
Note: This will have no impact if delete.topic.enable is not set to true.
Topic streamPerHourAlert2 is marked for deletion.
Note: This will have no impact if delete.topic.enable is not set to true.


## Create topics

In [156]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --create --replication-factor 1 --partitions 3 --topic rawevents --zookeeper "zk0-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk1-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk3-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181"
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --create --replication-factor 1 --partitions 3 --topic streamPerHourPerDay --zookeeper "zk0-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk1-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk3-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181"
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --create --replication-factor 1 --partitions 3 --topic streamPerHour --zookeeper "zk0-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk1-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk3-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181"
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --create --replication-factor 1 --partitions 3 --topic streamPerHourAlert2 --zookeeper "zk0-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk1-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk3-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181"

Created topic "rawevents".
Created topic "streamPerHourPerDay".
Created topic "streamPerHour".
Created topic "streamPerHourAlert2".


## Delete all checkpoints directory and output folders

In [157]:
%%bash
hdfs dfs -rm -r /path/checkpointstreamPerHour
hdfs dfs -rm -r /path/checkpointstreamPerHourPerDay
hdfs dfs -rm -r /path/chkpointalertType1query
hdfs dfs -rm -r /path/checkpointstreamPerHourAlert2
hdfs dfs -rm -r /path/chkpointalertType2queryNew
hdfs dfs -rm -r /example/AlertType1
hdfs dfs -rm -r /example/AlertType2

18/07/15 07:22:00 WARN azure.AzureFileSystemThreadPoolExecutor: Disabling threads for Rename operation as thread count 0 is <= 1
18/07/15 07:25:50 INFO azure.AzureFileSystemThreadPoolExecutor: Time taken for Rename operation is: 230670 ms with threads: 0
18/07/15 07:25:51 INFO fs.TrashPolicyDefault: Moved: 'wasb://sidsparkcluster-2018-07-13t05-21-51-393z@storesidkafkacluster.blob.core.windows.net/path/checkpointstreamPerHour' to trash at: wasb://sidsparkcluster-2018-07-13t05-21-51-393z@storesidkafkacluster.blob.core.windows.net/user/spark/.Trash/Current/path/checkpointstreamPerHour1531639312534
rm: `/path/checkpointstreamPerHourPerDay': No such file or directory
18/07/15 07:26:08 WARN azure.AzureFileSystemThreadPoolExecutor: Disabling threads for Rename operation as thread count 0 is <= 1
18/07/15 07:30:36 INFO azure.AzureFileSystemThreadPoolExecutor: Time taken for Rename operation is: 267772 ms with threads: 0
18/07/15 07:30:36 INFO fs.TrashPolicyDefault: Moved: 'wasb://sidsparkclust

## Create schemas

In [144]:
// The Kafka broker hosts and topic used to write to Kafka
val kafkaBrokers="wn0-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:9092,wn1-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:9092,wn2-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:9092"
val kafkaTopic="rawevents"
val zookeepers = "zk0-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk1-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181,zk3-kafka.vehan4uacj1epaynq40fn1fpke.cx.internal.cloudapp.net:2181"

// Define a schema for the data
val schema = (new StructType).
add("house_id", StringType).
add("eventtimestamp", StringType).
add("value", StringType).
add("household_id", StringType).
add("timestamp",StringType)

var kafkastreamPerHourPerDaySchema = (new StructType).
add("house_id", StringType).
add("household_id", StringType).
add("Date", DateType).
add("LastHour", StringType).
add("Id", StringType).
add("TotalConsumptionForThatHourAndDay",DoubleType).
add("timestamp",IntegerType).
add("window",(new StructType).
    add("start",TimestampType).
    add("end",TimestampType)
   )

var kafkastreamPerHourSchema = (new StructType).
add("house_id", StringType).
add("household_id", StringType).
add("LastHour", StringType).
add("MeanConsumptionForThatHour",DoubleType).
add("SDConsumptionForThatHour",DoubleType).
add("1SDConsumption", DoubleType).
add("timestamp",IntegerType).
add("window",(new StructType).
    add("start",TimestampType).
    add("end",TimestampType)
   )

var kafkastreamPerHourSchemaAlert2 = (new StructType).
add("house_id", StringType).
add("LastHour", StringType).
add("MeanConsumptionForThatHour",DoubleType).
add("SDConsumptionForThatHour",DoubleType).
add("1SDConsumption", DoubleType).
add("timestamp",IntegerType).
add("window",(new StructType).
    add("start",TimestampType).
    add("end",TimestampType)
   )

kafkastreamPerHourSchemaAlert2: org.apache.spark.sql.types.StructType = StructType(StructField(house_id,StringType,true), StructField(LastHour,StringType,true), StructField(MeanConsumptionForThatHour,DoubleType,true), StructField(SDConsumptionForThatHour,DoubleType,true), StructField(1SDConsumption,DoubleType,true), StructField(timestamp,IntegerType,true), StructField(window,StructType(StructField(start,TimestampType,true), StructField(end,TimestampType,true)),true))

## Create dataframes and queries

1. Create common dataframe on top of rawevents topic

In [145]:
var commonDF = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", "rawevents").
option("startingOffsets","latest").
option("failOnDataLoss","false").
load().
selectExpr("CAST(key AS STRING) AS key","CAST(value AS STRING) AS value").as[(String,String)].
select(from_json(col("value"), schema).alias("consumption")).
select("consumption.*").
selectExpr(
    "house_id as house_id",
    "household_id as household_id",
    "cast(cast(timestamp as int) as timestamp) as timestamp",
    "cast(timestamp as int) as timestamp_int",
    "cast(value as double) as consumption",
    "hour(from_unixtime(cast(eventtimestamp as int))) as LastHour",
    "cast(from_unixtime(cast(eventtimestamp as int)) as date) as Date"
).
withColumn("Id",concat(
    col("house_id"),lit("_"),
    col("household_id"),lit("_"),
    date_format(col("Date"),"dd-MM-yyyy"),lit("_"),
    col("LastHour"))).
withWatermark("timestamp","2 minutes").
dropDuplicates("Id")

commonDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [house_id: string, household_id: string ... 6 more fields]

## Start query to get hourly conumption for household for that hour for that day

In [146]:
var streamPerHourPerDay = commonDF.
groupBy(
    window($"timestamp", "60 minutes", "60 minutes"),
    col("house_id"),col("household_id"),col("Date"),col("LastHour"),col("Id")
).
agg(
    sum("consumption").as("TotalConsumptionForThatHourAndDay"),
    count("house_id").as("TotalEvents")
).
withColumn("timestamp",col("window.start").cast(IntegerType)).
selectExpr("cast(house_id as string) as key","to_json(struct(*)) AS value","timestamp").as[(String,String,Integer)].
writeStream.
format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("checkpointLocation", "/path/checkpointstreamPerHourPerDay1").
option("topic", "streamPerHourPerDay").
outputMode("append").
start()

/*format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("checkpointLocation", "/path/checkpointstreamPerHourPerDay").
option("topic", "streamPerHourPerDay").
outputMode("append").
start()

format("csv").
option("path","/example/streamPerHourPerDay").
option("checkpointLocation", "/path/checkpointstreamPerHourPerDay").
outputMode("append").
start() 
*/

streamPerHourPerDay: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@5e05fd79

In [120]:
println(streamPerHourPerDay.stop)


()

## Start query to get hourly conumption for a household for that hour from all past events

In [147]:
var streamPerHour = commonDF.
groupBy(
    window($"timestamp", "60 minutes", "60 minutes"),
    col("house_id"),col("household_id"),col("LastHour")
).
agg(
    mean("consumption").as("MeanConsumptionForThatHour")
    ,stddev_pop("consumption").as("SDConsumptionForThatHour")
).
withColumn(
    "1SDConsumption",col("MeanConsumptionForThatHour") + col("SDConsumptionForThatHour")
).
withColumn("timestamp",col("window.start").cast(IntegerType)).
selectExpr("cast(house_id as string) as key","to_json(struct(*)) AS value","timestamp").as[(String,String,Integer)].
writeStream.
format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("checkpointLocation", "/path/checkpointstreamPerHour").
option("topic", "streamPerHour").
outputMode("complete").
start()

/*
format("csv").
option("path","/example/streamPerHour").
option("checkpointLocation", "/path/checkpointstreamPerHour").
option("failOnDataLoss","false").
outputMode("complete").
start() 

format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("checkpointLocation", "/path/checkpointstreamPerHour").
option("topic", "streamPerHour").
outputMode("complete").
start()
*/

streamPerHour: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@77ff0ddb

In [115]:
streamPerHour.status

res15: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Getting offsets from KafkaSource[Subscribe[rawevents]]",
  "isDataAvailable" : false,
  "isTriggerActive" : true
}

## Start query to get hourly conumption for all households for that hour from all past events

In [148]:
var streamPerHourAlert2 = commonDF.
groupBy(
    window($"timestamp", "60 minutes", "60 minutes"),
    col("house_id"),col("LastHour")
).
agg(
    mean("consumption").as("MeanConsumptionForThatHour")
    ,stddev_pop("consumption").as("SDConsumptionForThatHour")
).
withColumn(
    "1SDConsumption",col("MeanConsumptionForThatHour") + col("SDConsumptionForThatHour")
).
withColumn("timestamp",col("window.start").cast(IntegerType)).
selectExpr("cast(house_id as string) as key","to_json(struct(*)) AS value","timestamp").as[(String,String,Integer)].
writeStream.
format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("checkpointLocation", "/path/checkpointstreamPerHourAlert2").
option("topic", "streamPerHourAlert2").
outputMode("complete").
start()

/*
format("csv").
option("path","/example/streamPerHourAlert2").
option("checkpointLocation", "/path/checkpointstreamPerHourAlert2").
option("failOnDataLoss","false").
outputMode("complete").
start() 

format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("checkpointLocation", "/path/checkpointstreamPerHourAlert2").
option("topic", "streamPerHourAlert2").
outputMode("complete").
start()
*/

streamPerHourAlert2: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@369c5097

In [149]:
streamPerHourAlert2.status

res40: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}

## Create dataframes and join queries
1. Consume date from streamPerHourPerDay topic
2. Consume date from streamPerHour topic
3. Consume date from streamPerHourAlert2 topic
4. Prepare a join condition for 1st adn 2nd dataframes for alert type 1 
5. Prepare a join condition for 1st adn 3rd dataframes for alert type 2

In [150]:
var kafkastreamPerHourPerDayDF = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", "streamPerHourPerDay").
option("startingOffsets","latest").
option("failOnDataLoss","false").
load().
selectExpr("CAST(key AS STRING) AS key","CAST(value AS STRING) AS value").as[(String,String)].
select(from_json(col("value"), kafkastreamPerHourPerDaySchema).alias("streamPerHourPerDay")).
select("streamPerHourPerDay.*").
select(
    col("house_id"),
    col("household_id"),
    col("LastHour"),
    col("Id"),
    col("window.start").as("Start"),
    col("window.end").as("End"),
    col("TotalConsumptionForThatHourAndDay"),
    col("timestamp").cast(TimestampType)
).
withWatermark("Start","5 minutes")

var kafkastreamPerHourDF = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", "streamPerHour").
option("startingOffsets","latest").
option("failOnDataLoss","false").
load().
selectExpr("CAST(key AS STRING) AS key","CAST(value AS STRING) AS value").as[(String,String)].
select(from_json(col("value"), kafkastreamPerHourSchema).alias("streamPerHour")).
select("streamPerHour.*").
select(
    col("house_id"),
    col("household_id"),
    col("LastHour"),
    col("window.start").as("Start"),
    col("window.end").as("End"),
    col("1SDConsumption"),
    col("MeanConsumptionForThatHour"),
    col("SDConsumptionForThatHour"),
    col("timestamp").cast(TimestampType)
).
withWatermark("Start","5 minutes")

var kafkastreamPerHourAlert2DF = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", "streamPerHourAlert2").
option("startingOffsets","latest").
option("failOnDataLoss","false").
load().
selectExpr("CAST(key AS STRING) AS key","CAST(value AS STRING) AS value").as[(String,String)].
select(from_json(col("value"), kafkastreamPerHourSchemaAlert2).alias("streamPerHour")).
select("streamPerHour.*").
select(
    col("house_id"),
    col("LastHour"),
    col("window.start").as("Start"),
    col("window.end").as("End"),
    col("1SDConsumption"),
    col("MeanConsumptionForThatHour"),
    col("SDConsumptionForThatHour"),
    col("timestamp").cast(TimestampType)
).
withWatermark("Start","5 minutes")

var finalResultAlert1 = kafkastreamPerHourPerDayDF.as("x").join(
    kafkastreamPerHourDF.as("y"),
    expr("""
        x.house_id = y.house_id AND
        x.household_id = y.household_id AND
        x.LastHour = y.LastHour AND
        x.Start >= y.Start AND x.Start <= y.End AND
        TotalConsumptionForThatHourAndDay > 1SDConsumption
        """),
    "leftOuter"
).
select(col("x.Id"),col("x.house_id"),col("x.household_id"),col("x.LastHour"),
       col("x.TotalConsumptionForThatHourAndDay"),
       col("y.1SDConsumption"),
       col("y.MeanConsumptionForThatHour"),
       col("y.SDConsumptionForThatHour")
      )

var finalResultAlert2 = kafkastreamPerHourPerDayDF.as("x").join(
    kafkastreamPerHourAlert2DF.as("y"),
    expr("""
        x.house_id = y.house_id AND
        x.LastHour = y.LastHour AND
        x.Start >= y.Start AND x.Start <= y.End AND
        TotalConsumptionForThatHourAndDay > 1SDConsumption
        """),
    "leftOuter"
).
select(col("x.Id"),col("x.house_id"),col("x.LastHour"),
       col("x.TotalConsumptionForThatHourAndDay"),
       col("y.1SDConsumption"),
       col("y.MeanConsumptionForThatHour"),
       col("y.SDConsumptionForThatHour")
      )

finalResultAlert2: org.apache.spark.sql.DataFrame = [Id: string, house_id: string ... 5 more fields]

## Generate alert type 1

In [151]:
var alertType1query = finalResultAlert1.
writeStream.
format("csv").
option("path","/example/AlertType1").
option("checkpointLocation", "/path/chkpointalertType1query").
option("failOnDataLoss","false").
trigger(Trigger.ProcessingTime("5 minutes")).
outputMode("append").
start() 

alertType1query: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@5dd3eb67

In [113]:
alertType1query.status

res13: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}

## Generate alert type 2

In [152]:
var alertType2query = finalResultAlert2.
writeStream.
format("csv").
option("path","/example/AlertType2").
option("checkpointLocation", "/path/chkpointalertType2queryNew").
option("failOnDataLoss","false").
trigger(Trigger.ProcessingTime("5 minutes")).
outputMode("append").
start() 

alertType2query: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6bbe4869

In [112]:
alertType2query.status

res12: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}

In [140]:
alertType1query.stop
streamPerHourPerDay.stop
streamPerHour.stop
streamPerHourAlert2.stop
alertType2query.stop